*Геллерт Анастасия Евгеньевна*

*Группа: 24.Б82-мм*

<center><h2>Доклад: Инструменты безопасности кода</h2></center>

**Инструменты безопасности кода** — это специализированные программы, которые помогают находить и устранять уязвимости и ошибки в программном обеспечении на ранних этапах его разработки. 

<u>**Категории инструментов:**</u>

**Static Application Security Testing(SAST)**\
Статический анализ кода

Анализирует исходный код (или байт-код) приложения до его запуска. Читает код и ищет шаблоны, которые соответствуют известным уязвимостям (например, SQL-инъекции, XSS, переполнение буфера).

Используется на ранних этапах разработки, часто интегрируется в среду разработки (IDE) или процесс непрерывной интеграции (CI/CD).

**Dynamic Application Security Testing(DAST)**\
Динамический анализ кода

Динамический анализ кода (DAST - Dynamic Application Security Testing)
Анализирует работающее приложение извне, имитируя атаки злоумышленника. Не видит исходный код, а отправляет различные запросы на работающее приложение и анализирует ответы.

Используется на поздних этапах разработки или тестирования, когда есть собранная и запущенная версия приложения.

**Software Composition Analysis(SCA)**\
Анализ компонентов

Сканирует приложение и его зависимости (библиотеки, фреймворки), чтобы составить список всех сторонних компонентов. Затем проверяет их на наличие известных уязвимостей по специальным базам данных.

Используется на любом этапе, но особенно важно на этапе сборки и при обновлении зависимостей.

____________________________________

**Bandit**

Инструмент с открытым исходным кодом, написанный на Python.\
Помогает анализировать Python-код и находить в нём наиболее распространённые уязвимости\
Является статическим анализатором безопасности (SAST)

<u>Помогает находит уязвимости:</u>
1. Командная инъекция\
   Когда программа принимает ввод от пользователя и передает его напрямую в системную команду, хакер может ввести не просто ожидаемые данные, а специальные символы и команды, тем самым выполнить любую команду на вашем сервере. 
2. SQL-инъекция\
   Когда SQL-запросы формируются путем простого склеивания строк с пользовательским вводом, хакер может изменить логику запроса.
3. Небезопасная десериализация\
   Десериализация — это обратный процесс сериализации, представляющий собой преобразование данных из линейного формата (например, последовательность байтов или символов) обратно в структуру объекта.\
   Если злоумышленник подменяет эти данные, он может создать специально сформированный объект, который при десериализации выполнит любой код на сервере.
4. XSS (Межсайтовый скриптинг)\
   Когда сайт отображает пользовательский контент без проверки, хакер может вставить вредоносный код. Этот код выполняется в браузерах других пользователей, позволяя воровать их сессии, перенаправлять на фишинговые сайты или совершать действия от их имени.
5. Path Traversal (Обход путей)
   Когда программа использует пользовательский ввод для формирования путей к файлам. Хакер может использовать специальные последовательности вроде "../", чтобы "выйти" из разрешенной директории и получить доступ к системным файлам, конфигурациям, паролям или личным данным других пользователей.\
\
и т.д.
   
   

Bandit нашёл применение в двух основных сферах:

1. Как один из процессов Continuous Integration (CI).
2. Как часть локального инструментария разработчика, используется для проверки кода на уязвимость до коммита.

In [1]:
#Установка
!pip install bandit

In [19]:
#Проверка установки и версии
!bandit --version 

bandit 1.8.6
  python version = 3.10.7 (tags/v3.10.7:6cc6b13, Sep  5 2022, 14:08:36) [MSC v.1933 64 bit (AMD64)]


In [ ]:
bandit code.py

In [ ]:
# Рекурсивное сканирование всей текущей директории и ее поддиректорий
!bandit -r .

# Сканирование конкретной папки в проекте
!bandit -r ./src

In [ ]:
#-x исключит определенные файлы или каталоги из проверки
bandit -r . -x some_directory/

Bandit создаст отчет с указанием всех найденных потенциальных проблем безопасности.\
По умолчанию Bandit покажет уровни уязвимости (LOW, MEDIUM и HIGH)

In [ ]:
#Не включает в отчет проблемы низкой степени серьезности.
bandit -r . -ll

<u>Пример работы:</u>

In [35]:
test1 = '''
import subprocess

domain = input("Enter the Domain: ")
output = subprocess.check_output(f"nslookup {domain}", shell=True, encoding='UTF-8')
print(output)
'''

with open('test1.py', 'w') as f:
    f.write(test1)

In [36]:
!bandit test1.py -ll

Run started:2025-10-10 19:02:46.988381

Test results:
>> Issue: [B602:subprocess_popen_with_shell_equals_true] subprocess call with shell=True identified, security issue.
   Severity: High   Confidence: High
   CWE: CWE-78 (https://cwe.mitre.org/data/definitions/78.html)
   More Info: https://bandit.readthedocs.io/en/1.8.6/plugins/b602_subprocess_popen_with_shell_equals_true.html
   Location: .\test1.py:5:9
4	domain = input("Enter the Domain: ")
5	output = subprocess.check_output(f"nslookup {domain}", shell=True, encoding='UTF-8')
6	print(output)

--------------------------------------------------

Code scanned:
	Total lines of code: 4
	Total lines skipped (#nosec): 0
	Total potential issues skipped due to specifically being disabled (e.g., #nosec BXXX): 0

Run metrics:
	Total issues (by severity):
		Undefined: 0
		Low: 1
		Medium: 0
		High: 1
	Total issues (by confidence):
		Undefined: 0
		Low: 0
		Medium: 0
		High: 2
Files skipped (0):


[main]	INFO	profile include tests: None
[main]	INFO	profile exclude tests: None
[main]	INFO	cli include tests: None
[main]	INFO	cli exclude tests: None
[main]	INFO	running on Python 3.10.7


Confidence: High потому что:
1. Есть прямой пользовательский ввод (input)
2. Есть shell=True
3. Есть конкатенация строк\
Создается ситуация уязвимости (командная инъекция)

In [38]:
test1 = '''
import subprocess

domain = input("Enter the Domain: ")
output = subprocess.check_output(["nslookup", domain], encoding='UTF-8')
print(output)
'''

with open('test1.py', 'w') as f:
    f.write(test1)

In [39]:
!bandit test1.py -ll

Run started:2025-10-10 19:03:29.589027

Test results:
	No issues identified.

Code scanned:
	Total lines of code: 4
	Total lines skipped (#nosec): 0
	Total potential issues skipped due to specifically being disabled (e.g., #nosec BXXX): 0

Run metrics:
	Total issues (by severity):
		Undefined: 0
		Low: 3
		Medium: 0
		High: 0
	Total issues (by confidence):
		Undefined: 0
		Low: 0
		Medium: 0
		High: 3
Files skipped (0):


[main]	INFO	profile include tests: None
[main]	INFO	profile exclude tests: None
[main]	INFO	cli include tests: None
[main]	INFO	cli exclude tests: None
[main]	INFO	running on Python 3.10.7


________

**Секрет-сканеры**(Специализированный SAST)

— это специализированные инструменты, которые автоматически обнаруживают случайно или намеренно оставленные в коде конфиденциальные данные. Они сканируют репозитории на наличие цифровых "ключей" от ваших сервисов и систем.

<u>Основные категории обнаруживаемых секретов:</u>

1. Ключи доступа к облачным провайдерам
2. API-ключи и токены сервисов
3. Ключи шифрования и сертификаты
4. Учетные данные баз данных
5. Конфигурационные файлы с секретами

**Gitleaks**

Нацелен на обнаружение секретов в истории Git репозитория.

1. Сканирует весь Git-репозиторий, включая историю коммитов
2. Использует предопределенные наборы правил для распространенных типов секретов.
3. Может быть интегрирован в CI/CD, чтобы проверять каждый коммит и пул-реквест.


**detect-secrets**

Нацелен на предотвращение попадания новых секретов в репозиторий.

1. Смотрит текущее состояния репозитория (игнорируя уже существующие секреты) и затем проверяет каждое новое изменение на наличие секретов.
2. Предназначен для использования в pre-commit хуках и CI.

**git-secrets**

Нацелен на предотвращение коммитов с секретами путем проверки на этапе pre-commit и pre-merge.

1. Устанавливает хуки в локальный Git репозиторий, которые автоматически проверяют изменения на наличие секретов перед коммитом и перед слиянием.
2. Проверяет как добавляемые файлы, так и коммиты, которые собираются быть замержены.
3. Позволяет добавлять пользовательские паттерны для специфичных для проекта секретов.

<u>Пример работы:</u>

In [59]:
# Устанавка
!pip install detect-secrets

In [60]:
!detect-secrets --version 

1.5.0


In [61]:
test2 = 'EMAIL_PASSWORD = "super_secret_123"'

with open('test2.py', 'w') as f:
    f.write(test2)

In [62]:
!detect-secrets scan test2.py

{
  "version": "1.5.0",
  "plugins_used": [
    {
      "name": "ArtifactoryDetector"
    },
    {
      "name": "AWSKeyDetector"
    },
    {
      "name": "AzureStorageKeyDetector"
    },
    {
      "name": "Base64HighEntropyString",
      "limit": 4.5
    },
    {
      "name": "BasicAuthDetector"
    },
    {
      "name": "CloudantDetector"
    },
    {
      "name": "DiscordBotTokenDetector"
    },
    {
      "name": "GitHubTokenDetector"
    },
    {
      "name": "GitLabTokenDetector"
    },
    {
      "name": "HexHighEntropyString",
      "limit": 3.0
    },
    {
      "name": "IbmCloudIamDetector"
    },
    {
      "name": "IbmCosHmacDetector"
    },
    {
      "name": "IPPublicDetector"
    },
    {
      "name": "JwtTokenDetector"
    },
    {
      "name": "KeywordDetector",
      "keyword_exclude": ""
    },
    {
      "name": "MailchimpDetector"
    },
    {
      "name": "NpmDetector"
    },
    {
      "name": "OpenAIDetector"
    },
    {
      "name": "Private

Устранить утечку:
1. Удалить секрет из кода и всех мест, куда он мог попасть.
2. Затем переместите его в безопасное место, например, в переменные окружения, специализированные хранилища секретов или защищенные конфигурационные файлы, которые не отслеживаются в Git.
3. Зачистите историю Git

In [65]:
test2 = '''
import os

email_password = os.environ.get('EMAIL_PASSWORD')
'''

with open('test2.py', 'w') as f:
    f.write(test2)

In [68]:
!detect-secrets scan test2.py

{
  "version": "1.5.0",
  "plugins_used": [
    {
      "name": "ArtifactoryDetector"
    },
    {
      "name": "AWSKeyDetector"
    },
    {
      "name": "AzureStorageKeyDetector"
    },
    {
      "name": "Base64HighEntropyString",
      "limit": 4.5
    },
    {
      "name": "BasicAuthDetector"
    },
    {
      "name": "CloudantDetector"
    },
    {
      "name": "DiscordBotTokenDetector"
    },
    {
      "name": "GitHubTokenDetector"
    },
    {
      "name": "GitLabTokenDetector"
    },
    {
      "name": "HexHighEntropyString",
      "limit": 3.0
    },
    {
      "name": "IbmCloudIamDetector"
    },
    {
      "name": "IbmCosHmacDetector"
    },
    {
      "name": "IPPublicDetector"
    },
    {
      "name": "JwtTokenDetector"
    },
    {
      "name": "KeywordDetector",
      "keyword_exclude": ""
    },
    {
      "name": "MailchimpDetector"
    },
    {
      "name": "NpmDetector"
    },
    {
      "name": "OpenAIDetector"
    },
    {
      "name": "Private

___

**Поверка зависимостей(SCA)**

**pip-audit**

— официальный инструмент от Python Packaging Authority (PyPA) для аудита безопасности зависимостей

1. Сканирует все установленные пакеты в окружении
2. Сверяется с базами уязвимостей (PyPI Advisory Database, OSV)
3. Показывает список уязвимых пакетов и доступные исправления

In [1]:
#Установка
!pip install pip-audit

In [ ]:
#Аудит текущего окружения
pip-audit

#Аудит файла
pip-audit -r <файл>
pip-audit -r requirements.txt

#Автоматическое исправление
pip-audit --fix

#Изменение формата вывода
pip-audit -f <формат>
pip-audit -f json

#Игнорирование уязвимости
pip-audit --ignore-vuln <ID>
pip-audit --ignore-vuln PYSEC-2021-123

<u>Пример работы</u>

In [4]:
import os
os.environ['PYTHONUTF8'] = '1'

In [5]:
test3 = 'django==1.11.29'

with open('test3.txt', 'w', encoding='utf-8') as f:
    f.write(test3)

In [6]:
!pip-audit -r test3.txt

- Collecting inputs
- Collecting inputs
- Collecting inputs
- Collecting inputs
\ Collecting inputs
\ Collecting inputs
\ Collecting inputs
\ Collecting inputs
| Collecting inputs
| Collecting inputs
| Collecting inputs
/ Collecting inputs
/ Collecting inputs
- Collecting inputs
- Collecting inputs
- Collecting inputs
\ Collecting inputs
\ Collecting inputs
\ Collecting inputs
| Collecting inputs
| Collecting inputs
| Collecting inputs
/ Collecting inputs
/ Collecting inputs
- Collecting inputs
- Collecting inputs
- Collecting inputs
\ Collecting inputs
\ Collecting inputs
\ Collecting inputs
| Collecting inputs
| Collecting inputs
| Collecting inputs
/ Collecting inputs
/ Collecting inputs
/ Collecting inputs
- Collecting inputs
- Collecting inputs
- Collecting inputs
\ Collecting inputs
\ Collecting inputs
| Collecting inputs
| Collecting inputs
| Collecting inputs
/ Collecting inputs
/ Collecting inputs
/ Collecting inputs
- Collecting inputs
- Collecting inputs
- Collecting inputs


  Requested location: "C:\Users\C2D6~1\AppData\Local\Temp\tmp88vut0q3\Scripts\python.exe"
  Actual location:    "C:\Users\РђСЃСЏ\AppData\Local\Temp\tmp88vut0q3\Scripts\python.exe"
Found 5 known vulnerabilities in 1 package


In [12]:
!pip-audit -r test3.txt --fix

- Collecting inputs
- Collecting inputs
- Collecting inputs
\ Collecting inputs
\ Collecting inputs
\ Collecting inputs
| Collecting inputs
| Collecting inputs
| Collecting inputs
/ Collecting inputs
/ Collecting inputs
/ Collecting inputs
- Collecting inputs
- Collecting inputs
\ Collecting inputs
\ Collecting inputs
\ Collecting inputs
| Collecting inputs
| Collecting inputs
| Collecting inputs
/ Collecting inputs
/ Collecting inputs
/ Collecting inputs
- Collecting inputs
- Collecting inputs
- Collecting inputs
\ Collecting inputs
\ Collecting inputs
\ Collecting inputs
| Collecting inputs
| Collecting inputs
/ Collecting inputs
/ Collecting inputs
/ Collecting inputs
- Collecting inputs
- Collecting inputs
- Collecting inputs
\ Collecting inputs
\ Collecting inputs
\ Collecting inputs
| Collecting inputs
| Collecting inputs
/ Collecting inputs
/ Collecting inputs
/ Collecting inputs
/ Collecting inputs
- Collecting inputs
- Collecting inputs
- Collecting inputs
\ Collecting inputs


  Requested location: "C:\Users\C2D6~1\AppData\Local\Temp\tmpkp1hwvpc\Scripts\python.exe"
  Actual location:    "C:\Users\РђСЃСЏ\AppData\Local\Temp\tmpkp1hwvpc\Scripts\python.exe"
Found 5 known vulnerabilities in 1 package and fixed 5 vulnerabilities in 1 package


In [13]:
with open('test3.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    print(content)

django==4.2.25



**Safety**

— это инструмент для проверки зависимостей на известные уязвимости в безопасности.\
В отличие от pip-audit, который использует официальную базу PyPA, использует собственную базу данных от pyup.io

In [2]:
#Установка
!pip install safety

In [ ]:
# Аудит текущего окружения
safety scan

# Аудит файла
safety scan -r requirements.txt

# Изменение формата вывода
safety scan --json